# Benchmark for d-wave

In [ ]:
from typing import Optional

In [ ]:
import matplotlib.pylab as plt
import os
import numpy as np
import pandas as pd

In [ ]:
from networkx import Graph
from networkx.algorithms.approximation import min_weighted_dominating_set

In [ ]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [ ]:
dwave_config = {
    # "solver": "DW_2000Q_5",
    "annealing_time": 100,  # integer microseconds [1, 2000]
    "answer_mode": "raw",
    "auto_scale": False,
    "num_reads": 100,  # raw will dump out all results
    "num_spin_reversal_transforms": 10,  # ask Travis what this is
}
sampler = DWaveSampler(solver="DW_2000Q_5")
embed = EmbeddingComposite(sampler)

In [ ]:
from qlp.mds import graph_tools as gt
from qlp.mds.qubo import get_mds_qubo
from qlp.mds.solver import classical_search

In [ ]:
def agg_res(df: pd.DataFrame, n_nodes: Optional[int] = None) -> pd.DataFrame:
    """Aggregate solution columns, find probablity and sort df
    """
    cols = [col for col in df.columns if str(col).isnumeric()]
    dist = (
        df.groupby(cols)
        .agg({"num_occurrences": "count", "energy": "mean"})
        .sort_values("num_occurrences", ascending=False)
    )
    if n_nodes is not None:
        not_cols = [col for col in cols if int(col) >= n_nodes]
        dist = dist.reset_index(not_cols).drop(columns=not_cols)

    dist["p"] = dist["num_occurrences"] / dist["num_occurrences"].sum()
    return dist

# Examples

## Most simple graph with unique sol

In [ ]:
graph = {(1, 0,), (2, 0)}

gt.get_plot(graph)
plt.show()

In [ ]:
qubo = get_mds_qubo(graph)
qubo.todense()

### Solve problem on D-Wave

In [ ]:
e, nodes = classical_search(qubo)
print("Brute force search classical minimal energy", e)
gt.get_plot(graph, nodes[0])
plt.show()

Apparently I had to manually rescale the QUBO to solve the right problem. It seems like min(Q) > -0.8 is a good setting

https://docs.dwavesys.com/docs/latest/c_qpu_3.html#adjusting-the-problem-scale

In [ ]:
q_min = np.array(list(qubo.values())).min()
fact = np.abs(q_min) * 0.8
print(fact)
qubo_rescaled = qubo / fact

Convert to dict 

In [ ]:
file_name = "graph_res_1-0-2.csv"
quobo_dict = {
    key: val for key, val in zip(qubo_rescaled.keys(), qubo_rescaled.values())
}

if os.path.exists(file_name):
    print("Reading data")
    raw = pd.read_csv(file_name)
else:
    print("Submitting")
    result = embed.sample_qubo(quobo_dict, **dwave_config)
    raw = result.to_pandas_dataframe()
    raw.to_csv(path_or_buf=file_name, index=False)

df = raw.sort_values("energy", ascending=True)
df["energy"] *= fact
df.head()

In [ ]:
dist = agg_res(df, n_nodes=3)
dist.head()

In [ ]:
n_cols = 5
first_five = dist.head(n=n_cols)

fig, axs = plt.subplots(ncols=n_cols, nrows=1, squeeze=False, figsize=(10,3), dpi=100)

for ind, res, ax in zip(first_five.index, first_five.to_dict("records"), axs.flatten()):
    color_nodes = set([i for i, v in enumerate(ind) if v==1])
    gt.get_plot_mpl(graph, color_nodes=color_nodes, ax=ax)
    p, e = res["p"], res["energy"]
    ax.set_title(f"p={int(p*100):d}%, e={int(e)}")

fig.suptitle("N=100", y=1.1)

plt.subplots_adjust(wspace=1.0)
plt.show()

## More complicated graph

In [ ]:
graph = {(0, 2), (0, 3), (0, 5), (1, 2), (1, 4), (3, 4), (4, 5)}
gt.get_plot(graph)
plt.show()

In [ ]:
qubo = get_mds_qubo(graph)

In [ ]:
e, nodes = classical_search(qubo)
print("Brute force search classical minimal energy", e)
gt.get_plot(graph, nodes[0])
plt.show()

In [ ]:
q_min = np.array(list(qubo.values())).min()
fact = np.abs(q_min) * 0.8
print(fact)
qubo_rescaled = qubo / fact

In [ ]:
file_name = "graph_res_6.csv"
quobo_dict = {
    key: val for key, val in zip(qubo_rescaled.keys(), qubo_rescaled.values())
}

if os.path.exists(file_name):
    print("Reading data")
    raw = pd.read_csv(file_name)
else:
    print("Submitting")
    result = embed.sample_qubo(quobo_dict, **dwave_config)
    raw = result.to_pandas_dataframe()
    raw.to_csv(path_or_buf=file_name, index=False)

df = raw.sort_values("energy", ascending=True)
df["energy"] *= fact
df.head()

In [ ]:
dist = agg_res(df, n_nodes=6).sort_values("energy")
dist.head()

In [ ]:
n_cols = 5
first_five = dist.head(n=n_cols)

fig, axs = plt.subplots(ncols=n_cols, nrows=1, squeeze=False, figsize=(12,3), dpi=100)

for ind, res, ax in zip(first_five.index, first_five.to_dict("records"), axs.flatten()):
    color_nodes = set([i for i, v in enumerate(ind) if v==1])
    gt.get_plot_mpl(graph, color_nodes=color_nodes, ax=ax)
    p, e = res["p"], res["energy"]
    ax.set_title(f"p={int(p*100):d}%, e={int(e)}")

fig.suptitle(f"D-Wave solutions sorted by energy; p_max = {dist.p.max()}, N=100", y=1.1)

plt.subplots_adjust(wspace=0.1)
plt.show()

## Presentation graph

In [ ]:
graph = {(0, 1), (1, 2), (1, 3), (2, 4), (3, 4), (3, 5), (4, 6), (6, 5)}

gt.get_plot(graph)
plt.show()

In [ ]:
qubo = get_mds_qubo(graph)

In [ ]:
q_min = np.array(list(qubo.values())).min()
fact = np.abs(q_min) * 0.8
print(fact)
qubo_rescaled = qubo / fact

In [ ]:
file_name = "graph_res_pres.csv"
quobo_dict = {
    key: val for key, val in zip(qubo_rescaled.keys(), qubo_rescaled.values())
}

if os.path.exists(file_name):
    print("Reading data")
    raw = pd.read_csv(file_name)
else:
    print("Submitting")
    result = embed.sample_qubo(quobo_dict, **dwave_config)
    raw = result.to_pandas_dataframe()
    raw.to_csv(path_or_buf=file_name, index=False)

df = raw.sort_values("energy", ascending=True)
df["energy"] *= fact
df.head()

In [ ]:
dist = agg_res(df, n_nodes=6).sort_values("energy")
dist.head()

In [ ]:
n_cols = 5
first_five = dist.head(n=n_cols)

fig, axs = plt.subplots(ncols=n_cols, nrows=1, squeeze=False, figsize=(12,3), dpi=100)

for ind, res, ax in zip(first_five.index, first_five.to_dict("records"), axs.flatten()):
    color_nodes = set([i for i, v in enumerate(ind) if v==1])
    gt.get_plot_mpl(graph, color_nodes=color_nodes, ax=ax)
    p, e = res["p"], res["energy"]
    ax.set_title(f"p={int(p*100):d}%, e={int(e)}")

fig.suptitle(f"D-Wave solutions sorted by energy; p_max = {dist.p.max()}, N=100", y=1.1)

plt.subplots_adjust(wspace=0.1)
plt.show()